# 🦌 Demonstrate 📊 in ELK

This notebook shows how you can add visualization inside ELK diagrams.

In [ ]:
if __name__ == "__main__":
    %pip install -q -r requirements.txt

In [ ]:
import bqplot as bq
import bqplot.pyplot as plt
import ipywidgets as W
import numpy as np

import ipyelk
import ipyelk.elements as e

unselected_style = {
    "opacity": 0.3,
}
selected_style = {
    "opacity": 1,
}


def hist_node(title, mark, width=60, height=60, key=None):
    height = 150
    width = 2 * height

    scale_x = mark.scales["sample"]
    scale_y = mark.scales["count"]

    figure = plt.Figure(
        marks=[mark],
        layout={
            "width": str(width) + "px",
            "height": str(height) + "px",
            "margin": "0",
        },
        fig_margin={
            "top": 7,
            "bottom": 15,
            "left": 7,
            "right": 7,
        },
        animation_duration=500,
        scale_x=scale_x,
        scale_y=scale_y,
        axes=[
            bq.Axis(scale=scale_x, num_ticks=3),
            bq.Axis(scale=scale_y, num_ticks=0, orientation="vertical"),
        ],
    )

    plot = e.Node(
        layoutOptions={"skip": "1"},  # something to prevent the default layout options
        properties=e.NodeProperties(
            shape=e.shapes.Widget(
                widget=figure,
            ),
            hidden=True,
        ),
        width=width,
        height=height,
    )
    node = e.Node(
        key=key,
        labels=[
            e.Label(text=title),
        ],
    )
    node.add_child(plot, key="plot")
    return node, figure


def model(x, y):
    #     return x * y + x**2 + .5 * y

    return x * -y + x**2 + 0.5 * -y


def hist(sample=None):
    if sample is None:
        sample = np.array()
    return bq.marks.Hist(
        sample=sample,
        scales={
            "sample": bq.LinearScale(),
            "count": bq.LinearScale(),
        },
        selected_style=selected_style,
        unselected_style=unselected_style,
    )


def chart_diagram_example():
    def calc_z(change=None):
        x_sel = x_hist.selected
        y_sel = y_hist.selected
        if x_sel is None and y_sel is None:
            x_data = x_hist.sample
            y_data = y_hist.sample
            index = x_sel
        else:
            if x_sel is not None and y_sel is not None:
                index = np.intersect1d(x_sel, y_sel)
            elif x_sel is not None:
                index = x_sel
            else:
                index = y_sel

            x_data = x_hist.sample[index]
            y_data = y_hist.sample[index]

        z_hist.sample = model(x_data, y_data)
        scatter.selected = index
        scatter.x = x_hist.sample
        scatter.y = y_hist.sample

    #     scatter.color = z_hist.sample
    x_rv = np.random.uniform(low=1, high=10, size=[1000])
    y_rv = np.random.uniform(low=1, high=10, size=[1000])
    z_rv = model(x_rv, y_rv)

    x_hist = hist(x_rv)
    y_hist = hist(y_rv)
    z_hist = hist(z_rv)

    for h in (x_hist, y_hist):
        h.observe(calc_z, ("sample", "selected"))

    x_node, x_fig = hist_node(title="X", mark=x_hist)
    y_node, y_fig = hist_node(title="Y", mark=y_hist)
    z_node, z_fig = hist_node(title="Z", mark=z_hist)

    for f in (x_fig, y_fig):
        brush = bq.interacts.BrushIntervalSelector(scale=f.scale_x, marks=f.marks)
        f.interaction = brush

    # 2D Figure and Node
    scale_x = x_fig.scale_x
    scale_y = y_fig.scale_x
    scale_z = bq.ColorScale()  # z_fig.scale_x
    width = 400
    height = 400

    scatter = bq.Scatter(
        x=x_hist.sample,
        y=y_hist.sample,
        color=z_hist.sample,
        scales={
            "x": scale_x,
            "y": scale_y,
            "color": scale_z,
        },
        selected_style={"opacity": 0.8},
        unselected_style={"opacity": 0.05},
    )

    xy_fig = plt.Figure(
        marks=[scatter],
        animation_duration=500,
        scale_x=scale_x,
        scale_y=scale_y,
        layout={
            "width": str(width) + "px",
            "height": str(height) + "px",
            "margin": "0",
        },
        axes=[
            bq.Axis(
                scale=scale_x,
                num_ticks=3,
                label="X",
            ),
            bq.Axis(
                scale=scale_y,
                num_ticks=3,
                orientation="vertical",
                label="Y",
            ),
        ],
    )

    plot = e.Node(
        layoutOptions={"skip": "1"},  # something to prevent the default layout options
        properties=e.NodeProperties(
            shape=e.shapes.Widget(
                widget=xy_fig,
            ),
            hidden=True,
        ),
        width=width,
        height=height,
    )
    xy_node = e.Node(
        labels=[
            e.Label(text="XYZ"),
        ]
    )
    xy_node.add_child(plot, key="plot")

    # Building Root Elements
    root = e.Node(
        children=[
            x_node,
            y_node,
            z_node,
            xy_node,
        ],
        edges=[
            e.Edge(
                source=x_node,
                target=z_node,
            ),
            e.Edge(
                source=y_node,
                target=z_node,
            ),
            e.Edge(
                source=x_node,
                target=xy_node,
            ),
            e.Edge(
                source=y_node,
                target=xy_node,
            ),
        ],
    )
    return ipyelk.from_element(root=root)

In [ ]:
if __name__ == "__main__":
    elk = chart_diagram_example()
    display(elk)
    # display pipe status widget for details
    # display(elk.pipe)